In [1]:
import pandas as pd

In [2]:
git_commit = pd.read_csv('/kaggle/input/linux-kernel-git-revision-history/linux_kernel_git_revlog.csv')

git_commit.drop(columns=['author_timestamp', 'commit_utc_offset_hours', 'subject', 'author_id'], inplace=True)
git_commit['modifications'] = git_commit['n_additions'] + git_commit['n_deletions']

In [3]:
def split_string(string, delimiter):
    if delimiter in string:
        return string.split(delimiter)
    else:
        return [string]

def get_directories(filepath):
    directories = split_string(filepath, '/')
    directories.reverse()
    result = []
    for i in range(0, len(directories)):
        result.append(directories[i:])
    return result

In [4]:
git_commit['FilePaths'] = git_commit['filename'].map(lambda f: get_directories(f))
git_commit.drop(columns=['filename', 'n_additions', 'n_deletions'], inplace=True)

In [5]:
expanded_paths = git_commit.explode('FilePaths')
expanded_paths['filepath'] = expanded_paths['FilePaths'].map(lambda f: '/'.join(f))
modifications_per_commit = expanded_paths.groupby(['commit_hash', 'filepath'])['modifications'].sum()

modifications_per_commit

commit_hash                               filepath                       
00001880cd8faaa349fe2ebb158f7e0cd8026048  drivers                             4
                                          net/drivers                         4
                                          qmi_wwan.c/usb/net/drivers          4
                                          usb/net/drivers                     4
000025cfbb9e8bafd0a1fa84d8d83526d24bde42  arch                               86
                                                                             ..
ffffa4b502099a03b5215b814fdcd79fb4348854  devices.txt/power/Documentation    49
                                          power/Documentation                49
ffffeaf318bd8da036eb8eb784b025a9f829201b  kernel                             12
                                          locking/kernel                     12
                                          qrwlock.c/locking/kernel           12
Name: modifications, Length: 3593847, dtype: i

In [6]:
modifications_per_commit.reset_index()

modifications_per_commit

commit_hash                               filepath                       
00001880cd8faaa349fe2ebb158f7e0cd8026048  drivers                             4
                                          net/drivers                         4
                                          qmi_wwan.c/usb/net/drivers          4
                                          usb/net/drivers                     4
000025cfbb9e8bafd0a1fa84d8d83526d24bde42  arch                               86
                                                                             ..
ffffa4b502099a03b5215b814fdcd79fb4348854  devices.txt/power/Documentation    49
                                          power/Documentation                49
ffffeaf318bd8da036eb8eb784b025a9f829201b  kernel                             12
                                          locking/kernel                     12
                                          qrwlock.c/locking/kernel           12
Name: modifications, Length: 3593847, dtype: i